# MoPy Design Document

This design document outlines the MoPy high level framework. It highlights what how we anticipate the user to interact with MoPy and provides a talking point and reference. 

A link to minutes from each day of the meeting, outlines discussion and outcomes:

https://docs.google.com/document/d/1c_OaG1QaZWoOYvkM2p1fNcpk3k4g8kMr2e_vrEfFrTA/edit?usp=sharing

In [ ]:
from obspy.clients.fdsn import Client 
import mopy

# MoPy will require some inputs at minimum in order to calculate Magnitudes.

## Waveforms

You can initialize an event client (ObsPy) or point to a waveform directory. This will make use of ObsPlus's wavebank feature.

## Event catalog (ObsPy)  

It contains at minium the event origin information i.e. lat (deg), lon (deg), depth (m) (relative to sea level - or at least the same datum of the velocity model), orgin time (UTC). 

***obspy.read_events will read many arbitrary catalog formats***   

## Station Metadata (ObsPy - Inventory) 

Inventory includes the station lat, lons, elev and instrument response info (time dependent).

## Windows (custom MoPy class)

This class contains the time windows that will be used to convert to the frequency domain. We decided to allow the user to define custom windows and just pass those through manually should they wish. The class will also have some basic ability to create basic time windows as shown in the code. The user will label the phase and many arbitrary phases may be passed through for simulataneous computation. 

## Source Velocity (scalar) - not needed

At least a constant value must be passed for velocity at the source. Multiple source velocities may be passed, but must be passed in the same order as the windows. 

***If this is not passed MoPy will look at .mopy.py for custom functions to define source velocity (to allow arbitrary 1/2/3/4D models)***

## Source Model

In order to obtain parameters for source modeling, we need to specifiy source models to fit the spectra too. We will have some built-in models commonly used for spectral modeling i.e. Brune, Boatright. However we also allow the user to specify an arbitrary source model. We use scipy's optimisation toolkit, so the only requirement is that the function should be created with this in mind.


In [ ]:
# Initialize a client to use to download waveforms
client = Client('IRIS')
# Read a local catalog e.g. QuakeML
cat = obspy.read_events('path_to_catalog')
# Read a local station inventory file
inv = obspy.read_inventory('path_to_inventory')
# channel info stores everything in a relational database 
chan_info = mopy.ChannelInfo(waveforms=client, catalog=cat, inventory=inv)

Define phase picks and time windows for spectral analysis and others:
    1. Velocities 
    2. Source Density
    3. Geometric Spreading
    4. Free Surface Effect

In [ ]:
# Option 1 - define a dictionary
picks = {("P", "event1", "UU.TMU..ELZ"): UTCDateTime(2019, 1, 1, 0, 0, 0),
         ("S", "event1", "UU.TMU..ELZ"): UTCDateTime(2019, 1, 1, 0, 0, 0.5)}
chan_info.set_picks(picks)

# Option 2 - use a csv file
picks = pd.read_csv("my_picks.csv")
chan_info = chan_info.set_picks(picks)

# can set just time windows or define relative to picks in the same manner as above
chan_info.set_rel_time_windows(P=(0,1), S=(1,10), inplace=True)


twinds = {("P", "event1", "UU.TMU..ELZ"): (UTCDateTime(2019, 1, 1, 0, 0, 0), UTCDateTime(2019, 1, 1, 0, 0, 1))
         ("S", "event1", "UU.TMU..ELZ"): (UTCDateTime(2019, 1, 1, 0, 0, 1), UTCDateTime(2019, 1, 1, 0, 0, 10))}
chan_info = chan_info.set_abs_time_windows(twinds)

Pass waveforms, catalog, inventory, time/phase windows, source velocity, source model(s) to `mopy.fit_spectra`. Calls the low level abstractions and passes all arguments. Fits all spectra from all phases and using all models.

In [ ]:
# Define an arbitrary source model function
def  ArbSourceModel():
        return omega_O, f_c
    
specs = mopy.fit_spectra(chan_info, source_model=('brune', 'boatright', ArbSourceModel)

Returns a dataframe with all of the source parameters added to the dataframe returned in specs using all of the fitted parameters.

In [ ]:
mags = mopy.calc_mw(specs)

Save database to arbitrary format or add to a catalog object. Does some test to see if there is an existing database to append to. We can take advantage of the many database formats that pandas supports. We can also append to existing databases with each run. We may also add some of our features to the catalog object, which can be used in later ObsPy-based workflows.

In [ ]:
# add to QuakeML
newcat = mags.add_to_cat(cat, inplace=False, **kwargs) 
# append to local database
mags.to_database(path='path_to_storage', fmt='sql', if_exists='append', **kwargs)

# Defaults are stored in .defaults.py 
We want to streamline the user interface to MoPy as much as we can. We therefore will have a default signal processing workflow so a user can easily pick up and play. Power users may then experiment with different workflows and source models. Other more complicated abstractions may be changed in here.

Defaults include:
1. Signal processing workflow - Rotate? Pad Zeros? Taper? FFT? Multi-taper?
2. Complex velocity models (1/3/4D)
